In [19]:
!pip install gradio
!pip install openai

In [37]:
# STEP 1: Import Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import gradio as gr
import json
from datetime import datetime
import os
import openai

# STEP 2: Load Final Dataset
df = pd.read_csv("final_multimodal_dataset.csv")

# STEP 3: Normalize fatigue labels to 0-based indexing
unique_fatigue_levels = sorted(df["fatigue_level"].unique())
fatigue_mapping = {val: idx for idx, val in enumerate(unique_fatigue_levels)}
df["fatigue_level"] = df["fatigue_level"].map(fatigue_mapping)

# STEP 4: Prepare Inputs and Labels
columns_to_drop = [
    "timestamp_x", "timestamp_rounded", "emotion_label", "classification",
    "mood_score", "fatigue_level", "source_file_x", "source_file_y"
]

features = df.drop(columns=columns_to_drop, errors='ignore')
features = features.select_dtypes(include=[np.number])

if "hrv_ms" not in features.columns:
    if "hrv_sec" in df.columns:
        df["hrv_ms"] = df["hrv_sec"] * 1000
    else:
        df["hrv_ms"] = 50.0
    features["hrv_ms"] = df["hrv_ms"]

x = features.values.astype(np.float32)
y_mood = df["mood_score"].values.astype(np.float32)
y_fatigue = df["fatigue_level"].values.astype(np.int64)

# STEP 5: Define Model
class NeuroModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.dropout = nn.Dropout(0.3)
        self.reg_head = nn.Linear(16, 1)
        self.cls_head = nn.Linear(16, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        mood = self.reg_head(x)
        fatigue = self.cls_head(x)
        return mood, fatigue

# STEP 6: Train Model
x_train, x_test, y_train_mood, y_test_mood, y_train_fatigue, y_test_fatigue = train_test_split(
    x, y_mood, y_fatigue, test_size=0.2, random_state=42, stratify=y_fatigue
)
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(x_train), torch.tensor(y_train_mood), torch.tensor(y_train_fatigue)
)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

input_size = x.shape[1]
num_classes = len(np.unique(y_fatigue))
model = NeuroModel(input_size=input_size, num_classes=num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_reg = nn.MSELoss()
loss_cls = nn.CrossEntropyLoss()

for epoch in range(10):
    model.train()
    total_loss = 0
    for xb, yb_mood, yb_fatigue in train_loader:
        optimizer.zero_grad()
        mood_pred, fatigue_pred = model(xb.float())
        loss = loss_reg(mood_pred.squeeze(), yb_mood.float()) + loss_cls(fatigue_pred, yb_fatigue) * 1.5
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} - Loss: {total_loss:.4f}")

# Save final model
torch.save(model.state_dict(), "neurotouch_model.pt")

# STEP 7: GPT Summary Generator
openai.api_key = os.getenv("OPENAI_API_KEY")

def generate_gpt_summary(entry):
    mood = entry['mood_score']
    fatigue = entry['fatigue_level']
    screen_time = entry['inputs']['screen_time_min']
    valence = entry['inputs']['valence_score']
    hrv = entry['inputs']['hrv_ms']
    tone = "positive and energetic" if mood > 7 else ("mindful and reflective" if fatigue == 1 else "exhausted and subdued")

    prompt = f"""
    Write a short, friendly summary about the user's mental state today.
    - Mood Score: {mood}/10
    - Fatigue Level: {fatigue}
    - HRV: {hrv} ms
    - Screen Time: {screen_time} minutes
    - Valence: {valence}
    Use a tone that feels {tone}.
    """

    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=200,
        stop=["\n"]
    )
    return response.choices[0].message.content.strip()

# STEP 8: Gradio Inference Function
def predict_neurotouch(alpha, beta, theta, hrv_ms, tap_speed, swipe_velocity, tap_pressure, screen_time_min,
                       unlock_freq, avg_key_hold_time, avg_key_interval, typo_rate, valence_score, mood_prev,
                       attention_prev):
    with torch.no_grad():
        input_values = [
            alpha, beta, theta, hrv_ms, tap_speed, swipe_velocity, tap_pressure, screen_time_min, unlock_freq,
            avg_key_hold_time, avg_key_interval, typo_rate, valence_score, mood_prev, attention_prev
        ]
        input_tensor = torch.tensor([input_values], dtype=torch.float32)
        mood_pred, fatigue_pred = model(input_tensor)
        mood_score = float(mood_pred.item())
        mood_score = max(1.0, min(mood_score, 10.0))
        fatigue_level = int(torch.argmax(fatigue_pred).item())

        # --- Heuristic Corrections ---
        correction_notes = []

        if hrv_ms < 30:
            fatigue_level = 2
            mood_score = min(mood_score, 6.0)
            correction_notes.append("Low HRV: Set fatigue=2, cap mood to 6")
        elif hrv_ms < 40 and screen_time_min > 480:
            fatigue_level = max(fatigue_level, 2)
            mood_score = min(mood_score, 5.5)
            correction_notes.append("HRV < 40 & high screen time: Fatigue boosted")
        elif fatigue_level == 2 and hrv_ms >= 40 and screen_time_min < 400:
            fatigue_level = 1
            correction_notes.append("HRV ok & low screen time: fatigue downgraded")

        if fatigue_level == 0 and valence_score < 0.3 and mood_score <= 6 and screen_time_min > 300:
            fatigue_level = 1
            correction_notes.append("Low valence + screen time: Fatigue nudged to 1")

        if valence_score < 0.3 and mood_score > 8 and mood_prev <= 6:
            mood_score = min(mood_score, 6.5)
            correction_notes.append("Mood overestimated: reduced to 6.5")

        if hrv_ms < 50 and screen_time_min > 300 and valence_score < 0.3:
            fatigue_level = max(fatigue_level, 1)
            correction_notes.append("Combo screen time+low HRV+valence: Fatigue = 1")

        if fatigue_level == 2:
            feedback = "⚠️ High fatigue detected — take a break 🧘"
        elif mood_score > 7:
            feedback = "⚡ Mood is elevated, great energy today!"
        else:
            feedback = "🧩 Mixed signals — stay mindful."

        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "mood_score": round(mood_score, 2),
            "fatigue_level": fatigue_level,
            "feedback": feedback,
            "inputs": {
                "alpha": alpha, "beta": beta, "theta": theta, "hrv_ms": hrv_ms,
                "tap_speed": tap_speed, "swipe_velocity": swipe_velocity, "tap_pressure": tap_pressure,
                "screen_time_min": screen_time_min, "unlock_freq": unlock_freq,
                "avg_key_hold_time": avg_key_hold_time, "avg_key_interval": avg_key_interval,
                "typo_rate": typo_rate, "valence_score": valence_score,
                "mood_prev": mood_prev, "attention_prev": attention_prev
            },
            "corrections": correction_notes
        }

        with open("daily_log.json", "a") as f:
            f.write(json.dumps(log_entry) + "\n")

        summary = generate_gpt_summary(log_entry)

        return {
            "Predicted Mood Score (1–10)": round(mood_score, 2),
            "Fatigue Level (0–2)": fatigue_level,
            "Feedback": feedback,
            "Corrections Applied": correction_notes,
            "Summary": summary
        }



# STEP 9: Gradio UI
with gr.Blocks(title="NeuroTouch Cognitive State Estimator", theme=gr.themes.Soft()) as app:
    gr.Markdown("# 🧠 NeuroTouch Dashboard\nEstimate cognitive state using EEG, HRV, and phone usage.")

    with gr.Row():
        with gr.Column():
            alpha = gr.Number(label="Alpha Power", value=0.5)
            beta = gr.Number(label="Beta Power", value=0.2)
            theta = gr.Number(label="Theta Power", value=0.3)
            hrv = gr.Number(label="HRV (ms)", value=50)
            tap_speed = gr.Number(label="Tap Speed", value=0.5)
            swipe = gr.Number(label="Swipe Velocity", value=1.0)
            pressure = gr.Number(label="Tap Pressure", value=0.85)
            screen_time = gr.Number(label="Screen Time (min)", value=300)

        with gr.Column():
            unlocks = gr.Number(label="Unlock Frequency", value=100)
            hold = gr.Number(label="Avg Key Hold Time", value=0.2)
            interval = gr.Number(label="Avg Key Interval", value=0.27)
            typo = gr.Number(label="Typo Rate", value=0.05)
            valence = gr.Number(label="Valence Score", value=0.2)
            mood_prev = gr.Number(label="Mood Score (Prev)", value=6)
            attention_prev = gr.Number(label="Attention Score (Prev)", value=6)
            submit = gr.Button("🧠 Predict Mood & Fatigue")

    output = gr.JSON()
    submit.click(fn=predict_neurotouch, inputs=[
        alpha, beta, theta, hrv, tap_speed, swipe, pressure, screen_time,
        unlocks, hold, interval, typo, valence, mood_prev, attention_prev
    ], outputs=output)

app.launch(debug=True)


Epoch 1 - Loss: 1182.9760
Epoch 2 - Loss: 868.1306
Epoch 3 - Loss: 848.2549
Epoch 4 - Loss: 813.4877
Epoch 5 - Loss: 795.8335
Epoch 6 - Loss: 795.0853
Epoch 7 - Loss: 784.6788
Epoch 8 - Loss: 768.6074
Epoch 9 - Loss: 789.0066
Epoch 10 - Loss: 775.5497
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://21cb0957e65ba9e9bc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://21cb0957e65ba9e9bc.gradio.live
